### Imports and logging

First, we start with our imports and get logging established:

In [36]:
# imports needed and set up logging
import bz2
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [37]:
data_file="news.crawl.bz2"

with bz2.open ('news.crawl.bz2', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b'\xc2\xbf Robert J. Spagnoletti , attorney general : $ 22,903 * * \n'


### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [38]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with bz2.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2020-04-29 18:43:35,668 : INFO : reading file news.crawl.bz2...this may take a while
2020-04-29 18:43:35,691 : INFO : read 0 reviews
2020-04-29 18:43:36,096 : INFO : read 10000 reviews
2020-04-29 18:43:36,505 : INFO : read 20000 reviews
2020-04-29 18:43:36,875 : INFO : read 30000 reviews
2020-04-29 18:43:38,712 : INFO : read 40000 reviews
2020-04-29 18:43:39,082 : INFO : read 50000 reviews
2020-04-29 18:43:39,439 : INFO : read 60000 reviews
2020-04-29 18:43:39,798 : INFO : read 70000 reviews
2020-04-29 18:43:40,135 : INFO : read 80000 reviews
2020-04-29 18:43:40,480 : INFO : read 90000 reviews
2020-04-29 18:43:40,832 : INFO : read 100000 reviews
2020-04-29 18:43:41,180 : INFO : read 110000 reviews
2020-04-29 18:43:41,532 : INFO : read 120000 reviews
2020-04-29 18:43:41,868 : INFO : read 130000 reviews
2020-04-29 18:43:42,238 : INFO : read 140000 reviews
2020-04-29 18:43:42,570 : INFO : read 150000 reviews
2020-04-29 18:43:42,909 : INFO : read 160000 reviews
2020-04-29 18:43:43,293 : IN

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [39]:
model = gensim.models.Word2Vec (documents, size=150, window=5, min_count=2, workers=8, iter=10)

rds/s, in_qsize 13, out_qsize 2
2020-04-29 18:47:34,295 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-04-29 18:47:34,311 : INFO : EPOCH 5 - PROGRESS: at 99.87% examples, 1250613 words/s, in_qsize 6, out_qsize 1
2020-04-29 18:47:34,318 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-04-29 18:47:34,320 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-04-29 18:47:34,324 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-04-29 18:47:34,326 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-04-29 18:47:34,327 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-29 18:47:34,334 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-29 18:47:34,337 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-29 18:47:34,337 : INFO : EPOCH - 5 : training on 39198623 raw words (31594539 effective words) took 25.3s, 1250988 eff

Q1: Report similarity scores for the following pairs: (dirty,clean),(big,dirty),(big,large),(big,small)

In [50]:
model.wv.similarity('dirty','clean')

0.35809702

In [51]:
model.wv.similarity('big','dirty')

0.31531984

In [52]:
model.wv.similarity('big','large')

0.47316137

In [53]:
model.wv.similarity('big','small')

0.5379722

Q2: Report 5 most similar items and the scores to 'polite', 'orange'

In [54]:
w1 = ["polite"]
model.wv.most_similar(w1,topn=5)

[('respectful', 0.7054284811019897),
 ('gracious', 0.703324019908905),
 ('courteous', 0.6848858594894409),
 ('cheerful', 0.6522202491760254),
 ('thoughtful', 0.6481714248657227)]

In [55]:
w1 = ["orange"]
model.wv.most_similar(w1,topn=5)

[('yellow', 0.533903181552887),
 ('ventura', 0.5247911214828491),
 ('riverside', 0.49417170882225037),
 ('upturned', 0.48955097794532776),
 ('cuyahoga', 0.48696306347846985)]

Q3: Now change the parameters of your model as follows: window=2, size=50. Answer the 2 questions above for this new model.

In [56]:
model2 = gensim.models.Word2Vec (documents, size=50, window=2, min_count=2, workers=8, iter=10)

shed; awaiting finish of 6 more threads
2020-04-29 18:51:44,039 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-04-29 18:51:44,040 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-04-29 18:51:44,043 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-04-29 18:51:44,055 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-29 18:51:44,056 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-29 18:51:44,058 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-29 18:51:44,058 : INFO : EPOCH - 4 : training on 39198623 raw words (31595710 effective words) took 21.8s, 1451690 effective words/s
2020-04-29 18:51:45,085 : INFO : EPOCH 5 - PROGRESS: at 4.55% examples, 1429381 words/s, in_qsize 12, out_qsize 3
2020-04-29 18:51:46,098 : INFO : EPOCH 5 - PROGRESS: at 9.79% examples, 1530291 words/s, in_qsize 14, out_qsize 1
2020-04-29 18:51:47,101 : INFO : EPOCH 5 - PR

In [57]:
model2.wv.similarity('dirty','clean')

0.5236713

In [58]:
model2.wv.similarity('big','dirty')

0.4496475

In [59]:
model2.wv.similarity('big','large')

0.7133521

In [60]:
model2.wv.similarity('big','small')

0.7466693

In [61]:
w1 = ["polite"]
model2.wv.most_similar(w1,topn=5)

2020-04-29 18:53:49,618 : INFO : precomputing L2-norms of word weight vectors


[('respectful', 0.8081028461456299),
 ('timid', 0.7878763675689697),
 ('courteous', 0.7826941609382629),
 ('cheerful', 0.7705578804016113),
 ('candid', 0.7689613103866577)]

In [63]:
w1 = ["orange"]
model2.wv.most_similar(w1,topn=5)

[('blue', 0.731901228427887),
 ('poinsettia', 0.6719993948936462),
 ('upturned', 0.6449915170669556),
 ('calico', 0.6449728608131409),
 ('fringed', 0.6284209489822388)]